In [2]:
# Imports
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import re
import numpy as np
import helpers # this is a Joe G. created helper file of functions
from bertopic import BERTopic

In [209]:
# Params / Files to change 
input_file = '../data/cleaned_data/NLX Jobs.csv' # change to whatever file/filepath you are using
output_file = '../data/results/nlx/topic_match/all_v3.csv' # change to your outpath
model_file = '../data/results/nlx/topic_match/all_v3' # change to save model path (** NOTE no file ext!!! **)
desc_column = 'Description'
start_idx = -1 # file row to start at
end_idx = 1000 # file row to end at
#how many jobs do you want to search / score against? make start_idx -1 if you want to use entire file

In [7]:
# model name for BERTopic 
model_name = 'sentence-transformers/paraphrase-distilroberta-base-v2'
model = SentenceTransformer(model_name)

In [205]:
# file for our cyber baseline
baseline_file = '../data/cleaned_data/KUKSAT_Coded_Baseline.csv'
ksa_col = 'KUKSAT'

In [206]:
# read baselne_file and convert to df
df_base = pd.read_csv(baseline_file, encoding='ISO-8859-1')
ksas = df_base[ksa_col].tolist()

In [207]:
# how many ksas
len(df_base)

2263

In [210]:
# read input file and covert to df
# covert target description column to list
df = pd.read_csv(input_file)
if start_idx >= 0 and start_idx != end_idx:
    jobs = df[start_idx:end_idx]
else:
    jobs = df.copy()
jobs = jobs[desc_column].tolist()

In [211]:
# how many jobs
len(jobs)

12380

In [212]:
# apply text cleanup functions to jobs and ksa base list
jobs = helpers.remove_stopwords(jobs)
ksas = helpers.remove_stopwords(ksas)

jobs = helpers.cleanup_text(jobs)
ksas = helpers.cleanup_text(ksas)

In [66]:
# initalize model, make bigram, fit to job sents
topic_model = BERTopic(embedding_model=model, n_gram_range=(1, 2))

topics, probs = topic_model.fit_transform(jobs)

In [127]:
# view topics
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,3725,-1_security_information_systems_management
1,0,413,0_ia_assurance ia_information assurance_assurance
2,1,326,1_you_you develop_requirements you_systems you
3,2,307,2_cyberspace_cyberspace operations_operations_...
4,3,196,3_you_navy_you provide_naval
...,...,...,...
255,261,11,261_number_enforcement approved_broader busine...
269,268,10,268_test_specifications requirements_operation...
270,269,10,269_inspections_dss_industry classified_accred...
271,270,10,270_analyses collected_data findings_expert an...


In [68]:
# saves topic model
topic_model.save(model_file)

C:\Users\Joe\anaconda3\lib\site-packages\scipy\sparse\_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [215]:
# Try using each KSA and compare see the top 5 topics that fit
# If matches (x >= 0.6), similar matches ( 0.6< x >0.4)
all_matches = []
all_similar =[]
for idx, ksa in enumerate(ksas):
    matched=[]
    similar=[]
    topics, similarity = topic_model.find_topics(ksa,top_n=5)

    for idx2, score in enumerate(similarity):
        temp={}
        if score >= 0.6:
            temp['ksa_code'] = df_base['Type'][idx]
            temp['ksa_idx'] = idx
            temp['ksa_text'] = ksas[idx]
            temp['topic_idx'] = topics[idx2]
            temp['sim_score'] = score
            temp['topic_info'] = topic_model.get_topic(idx2)
            matched.append(temp)
        if score <0.6 and score > 0.4:
            temp['ksa_code'] = df_base['Type'][idx]
            temp['ksa_idx'] = idx
            temp['ksa_text'] = ksas[idx]
            temp['topic_idx'] = topics[idx2]
            temp['sim_score'] = score
            temp['topic_info'] = topic_model.get_topic(idx2)
            similar.append(temp)
    all_matches.append(matched)
    all_similar.append(similar)

In [216]:
# calculates aggregate scores for matching, similar, missing
# uses total topics
ksa_agg_matches = []
ksa_agg_similar = []
ksa_agg_missing = []
num_topics = len(topic_model.get_topic_info()) -1
for idx, val in enumerate(ksas):
    matched_score = len(all_matches[idx])/num_topics
    similar_score = len(all_similar[idx])/num_topics
    missing_score = 1 -(matched_score + similar_score)
    ksa_agg_matches.append(matched_score)
    ksa_agg_similar.append(similar_score)
    ksa_agg_missing.append(missing_score)

In [217]:
# dataframes all results
final_df = pd.DataFrame({
    'ksa': ksas,
    'matches': all_matches,
    'similar': all_similar,
    'matched_score': ksa_agg_matches,
    'similar_score': ksa_agg_similar,
    'missing_score': ksa_agg_missing
})

In [218]:
# save to output_file
final_df.to_csv(output_file, index=False)

In [219]:
# For local visual / spot check sorts by matched, similar, missing score
final_df.sort_values(by=['matched_score', 'similar_score', 'missing_score'], ascending=False)

,ksa,matches,similar,matched_score,similar_score,missing_score
442,Knowledge organization s threat environment.,"[{'ksa_code': 'K0344', 'ksa_idx': 442, 'ksa_te...","[{'ksa_code': 'K0344', 'ksa_idx': 442, 'ksa_te...",0.009836,0.006557,0.983607
45,People security social engineering,"[{'ksa_code': 'KU', 'ksa_idx': 45, 'ksa_text':...","[{'ksa_code': 'KU', 'ksa_idx': 45, 'ksa_text':...",0.006557,0.009836,0.983607
92,"Threat Information Sources e.g., CERT","[{'ksa_code': 'KU', 'ksa_idx': 92, 'ksa_text':...","[{'ksa_code': 'KU', 'ksa_idx': 92, 'ksa_text':...",0.006557,0.009836,0.983607
269,Knowledge current emerging threats threat vect...,"[{'ksa_code': 'K0151', 'ksa_idx': 269, 'ksa_te...","[{'ksa_code': 'K0151', 'ksa_idx': 269, 'ksa_te...",0.006557,0.009836,0.983607
283,Knowledge risk threat assessment.,"[{'ksa_code': 'K0165', 'ksa_idx': 283, 'ksa_te...","[{'ksa_code': 'K0165', 'ksa_idx': 283, 'ksa_te...",0.006557,0.009836,0.983607
...,...,...,...,...,...,...
2239,Define triggers within continuous monitoring p...,[],[],0.000000,0.000000,1.000000
2240,Establish scoring grading metrics measure effe...,[],[],0.000000,0.000000,1.000000
2244,Use non automated assessment methods data cont...,[],[],0.000000,0.000000,1.000000
2257,Using scores grades motivate assess performanc...,[],[],0.000000,0.000000,1.000000


In [220]:
len(final_df[final_df['missing_score'] !=1])

1889

In [221]:
len(final_df[final_df['missing_score'] !=1]) / len(ksas)

0.8347326557666814

In [20]:
# USA Jobs
# started 8:25pm
# ended at 8:46pm

In [21]:
# NLX 
# started 9:08 pm
# ended 9:31 pm

In [22]:
# topic_model=BERTopic.load('path_to_model', embedding_model=model)

In [3]:
import ast

In [72]:
load_results = '../data/results/nlx/topic_match/all_v3.csv'
final_df = pd.read_csv(load_results,converters={'matches':ast.literal_eval, 'similar': ast.literal_eval, 
                                               'missing': ast.literal_eval})

In [73]:
load_model  = '../data/results/nlx/topic_match/all_v2'

In [74]:
topic_model=BERTopic.load(load_model, embedding_model=model)

In [75]:
topic_df = topic_model.get_topic_info()
# topic_df = topic_df[topic_df['Count'] >= 50]

In [76]:
len(topic_df)

306

In [77]:
temp = pd.DataFrame()
for idx, val in final_df.iterrows():
    temp2 = pd.DataFrame(val['similar'])
    temp3 = pd.DataFrame(val['matches'])
    temp = pd.concat([temp, temp2,temp3])

In [78]:
len(list(set(temp['topic_idx'].tolist())))

165

In [88]:
# KSA count per Topic
topic_count = temp.groupby('topic_idx').size().reset_index(name='count').sort_values(by=['count'], ascending=False)
topic_count

,topic_idx,count
91,171,709
64,112,470
104,194,426
109,200,377
113,207,320
...,...,...
94,179,1
116,216,1
7,12,1
128,237,1


In [95]:
topic_matches = topic_count['topic_idx'].tolist()
doc_sum = 0
for i in topic_matches:
    print('TOPIC ', i)
#     print('TOPIC ', i, ' Count ', topic_model.get_topic_info(i)['Count'])
    print(topic_model.get_topic_freq(i))
    doc_sum += topic_model.get_topic_freq(i)

TOPIC  171
22
TOPIC  112
31
TOPIC  194
20
TOPIC  200
19
TOPIC  207
19
TOPIC  260
14
TOPIC  227
17
TOPIC  281
12
TOPIC  248
15
TOPIC  282
12
TOPIC  229
17
TOPIC  82
38
TOPIC  224
17
TOPIC  18
73
TOPIC  121
29
TOPIC  -1
1673
TOPIC  197
20
TOPIC  20
71
TOPIC  196
20
TOPIC  31
63
TOPIC  76
41
TOPIC  302
10
TOPIC  221
18
TOPIC  46
55
TOPIC  244
15
TOPIC  101
33
TOPIC  255
14
TOPIC  108
31
TOPIC  225
17
TOPIC  30
63
TOPIC  59
50
TOPIC  230
17
TOPIC  161
23
TOPIC  86
38
TOPIC  1
319
TOPIC  236
16
TOPIC  198
20
TOPIC  160
24
TOPIC  272
13
TOPIC  228
17
TOPIC  283
12
TOPIC  35
61
TOPIC  93
36
TOPIC  114
30
TOPIC  285
12
TOPIC  16
75
TOPIC  60
49
TOPIC  183
21
TOPIC  42
57
TOPIC  97
34
TOPIC  8
113
TOPIC  130
27
TOPIC  204
19
TOPIC  43
56
TOPIC  138
26
TOPIC  262
14
TOPIC  199
20
TOPIC  13
78
TOPIC  220
18
TOPIC  261
14
TOPIC  180
22
TOPIC  54
52
TOPIC  105
33
TOPIC  131
27
TOPIC  202
19
TOPIC  17
74
TOPIC  254
15
TOPIC  3
214
TOPIC  170
22
TOPIC  44
55
TOPIC  286
12
TOPIC  175
22
TOPIC  245
15


In [96]:
print(doc_sum)
print(topic_df['Count'].sum())

7624
12380


In [97]:
# percent of jobs that had a matching or similar ksa
doc_sum/topic_df['Count'].sum()

0.6158319870759289

In [227]:
good_topics = [171,112,194,200]
bad_topics=[2,5,9,12]

In [232]:
# generates bar charts per topic
bar_fig = topic_model.visualize_barchart(topics=good_topics, width=450, height=300)
bar_fig.write_html("../data/results/nlx/topic_match/bar_chart_good4_v3.html")
bar_fig

In [229]:
print('GOOD', topic_count[topic_count['topic_idx'].isin(good_topics) ])
print('BAD', topic_count[topic_count['topic_idx'].isin(bad_topics) ])

GOOD      topic_idx  count
91         171    709
64         112    470
104        194    426
109        200    377
BAD    topic_idx  count
7         12      1


In [233]:
for i in good_topics:
    frequency = topic_model.get_topic_freq(i)
    print(i, frequency)

171 22
112 31
194 20
200 19


In [90]:
len(final_df[final_df['missing_score'] < 1]) / len(final_df)

0.8347326557666814

In [235]:
for i in bad_topics:
    frequency = topic_model.get_topic_freq(i)
    print(i, frequency)

2 263
5 127
9 111
12 82


In [236]:
final_df.sort_values(by=['matched_score', 'similar_score'], ascending=False)

,ksa,matches,similar,matched_score,similar_score,missing_score
442,Knowledge organization s threat environment.,"[{'ksa_code': 'K0344', 'ksa_idx': 442, 'ksa_te...","[{'ksa_code': 'K0344', 'ksa_idx': 442, 'ksa_te...",0.009836,0.006557,0.983607
45,People security social engineering,"[{'ksa_code': 'KU', 'ksa_idx': 45, 'ksa_text':...","[{'ksa_code': 'KU', 'ksa_idx': 45, 'ksa_text':...",0.006557,0.009836,0.983607
92,"Threat Information Sources e.g., CERT","[{'ksa_code': 'KU', 'ksa_idx': 92, 'ksa_text':...","[{'ksa_code': 'KU', 'ksa_idx': 92, 'ksa_text':...",0.006557,0.009836,0.983607
269,Knowledge current emerging threats threat vect...,"[{'ksa_code': 'K0151', 'ksa_idx': 269, 'ksa_te...","[{'ksa_code': 'K0151', 'ksa_idx': 269, 'ksa_te...",0.006557,0.009836,0.983607
283,Knowledge risk threat assessment.,"[{'ksa_code': 'K0165', 'ksa_idx': 283, 'ksa_te...","[{'ksa_code': 'K0165', 'ksa_idx': 283, 'ksa_te...",0.006557,0.009836,0.983607
...,...,...,...,...,...,...
2239,Define triggers within continuous monitoring p...,[],[],0.000000,0.000000,1.000000
2240,Establish scoring grading metrics measure effe...,[],[],0.000000,0.000000,1.000000
2244,Use non automated assessment methods data cont...,[],[],0.000000,0.000000,1.000000
2257,Using scores grades motivate assess performanc...,[],[],0.000000,0.000000,1.000000


In [237]:
ksa_count = temp.groupby('ksa_idx').size().reset_index(name='count').sort_values(by=['count'], ascending=False)
ksa_count

,ksa_idx,count
944,1176,5
842,1057,5
1444,1750,5
848,1063,5
1445,1751,5
...,...,...
791,994,1
797,1003,1
798,1004,1
799,1006,1


In [238]:
temp.sort_values(by=['sim_score'], ascending=False)

,ksa_code,ksa_idx,ksa_text,topic_idx,sim_score,topic_info
0,T0423,1681,Analyze computer generated threats counter int...,260,0.711996,"[(va, 0.02300448158630805), (esa, 0.0202865459..."
0,T0169,1428,Perform cybersecurity testing developed applic...,281,0.708224,"[(va, 0.02300448158630805), (esa, 0.0202865459..."
0,T0708,1965,"Identify threat tactics, methodologies.",260,0.708145,"[(va, 0.02300448158630805), (esa, 0.0202865459..."
1,T0423,1681,Analyze computer generated threats counter int...,207,0.696913,"[(gdit, 0.0182916716344354), (vulnerability, 0..."
0,T0685,1942,Evaluate threat decision making processes.,260,0.695986,"[(va, 0.02300448158630805), (esa, 0.0202865459..."
...,...,...,...,...,...,...
3,K0123,242,Knowledge legal governance related admissibili...,112,0.400181,"[(ctg, 0.03028589370242274), (business, 0.0098..."
2,T0840,2096,Collaborate across internal and or external or...,228,0.400174,"[(nelnet, 0.029197850804991297), (that, 0.0115..."
2,T0978,2234,Work organizational officials establish system...,112,0.400093,"[(nelnet, 0.029197850804991297), (that, 0.0115..."
3,K0506,589,"Knowledge organization objectives, leadership ...",46,0.400026,"[(ctg, 0.03028589370242274), (business, 0.0098..."


In [86]:
topic_plot = topic_model.visualize_topics(width=800, height=800)
topic_plot.write_html("../data/results/nlx/topic_match/inter_topic_allv3.html")

In [87]:
topic_plot

In [ ]:
topic_model.visualize_topics(width=800, height=800)